In [9]:
end_date="2017-12-28"
till_date="2017-12-27"
def date_to_timestampMS(date_str):
    '''
    Args:
        date_str, eg. '2017-11-08'
    return:
        str, timestamp
    '''
    d = datetime.datetime.strptime(date_str, '%Y-%m-%d')
    return str(int((d-datetime.datetime(1970,1,1)).total_seconds()*1000))

In [10]:
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import datetime
import os

KEY_FILE_LOCATION = '/home/jian/Projects/LIFT/DBM/Key_files/dbm_api_jubapluscc@gmail.com.json'
SCOPES = ['https://www.googleapis.com/auth/doubleclickbidmanager']

credentials = ServiceAccountCredentials.from_json_keyfile_name(
    KEY_FILE_LOCATION, scopes=SCOPES)

# Build the service object.
service = build('doubleclickbidmanager', 'v1', credentials=credentials)

In [11]:
from urllib.request import urlopen
from contextlib import closing

def download_reports(query_ids, output_dir):
    '''
    Args: 
        query_ids: list of str
        output_dir: str, eg. '/home/jzou'
    '''
    for query_id in query_ids:
        query = service.queries().getquery(queryId=query_id).execute()
        report_url = query['metadata']['googleCloudStoragePathForLatestReport']
        output_file = '%s/%s.csv' % (output_dir, query['metadata']['title'])
        with open(output_file, 'wb') as output:
            with closing(urlopen(report_url)) as url:
                output.write(url.read())

In [12]:
import json
old_queries = json.load(open('/home/jian/Projects/LIFT/DBM/Key_files/queries_delete_unknow.json', 'r'))
old_titles = [x['metadata']['title'] for x in old_queries['queries']]
'''
old_titles.index('LIFT_Date_OCT29') #137
old_titles.index('LIFT_Weekday_OCT29') #144
old_titles.index('LIFT_Time_of_Day_OCT29') #143
old_titles.index('LIFT_City_OCT29') #134
old_titles.index('LIFT_DMA_OCT29') #139
old_titles.index('LIFT_Region_OCT29') #142
old_titles.index('LIFT_APP_URL_OCT29') #131
old_titles.index('LIFT_Browser_OCT29') #132
old_titles.index('LIFT_Creative_OCT29') #135
old_titles.index('LIFT_Creative_Size_Oct29') #136
old_titles.index('LIFT_Device_Type_OCT29') #138
old_titles.index('LIFT_ISP_Carrier_OCT29') #140
old_titles.index('LIFT_Operating_System_OCT29') #141
'''
old_index_list=[137,144,143,134,139,142,131,132,135,136,138,140,141]
parameters = [old_queries['queries'][x]['params'] for x in old_index_list]
old_title_list = [old_queries['queries'][x]['metadata']['title'].replace("_OCT29", "_to_"+str(end_date)) for x in old_index_list]

In [13]:
query_resourse_list=[]
for i in range(13):
    x={
        'kind': 'doubleclickbidmanager#query',
        'metadata': {
            'dataRange': 'CUSTOM_DATES',
            'format': 'CSV',
            'title': old_title_list[i]
        },
        'params':parameters[i],
        'reportDataEndTimeMs': date_to_timestampMS(end_date),
        'reportDataStartTimeMs': date_to_timestampMS("2017-09-25"),
        'schedule': {
            'endTimeMs': '0',
            'frequency': 'ONE_TIME',
            'nextRunTimezoneCode': 'America/New_York'
        }
    }
    query_resourse_list.append(x)


# create query, only need to run once

In [14]:
'''
query_resourse = {
    'kind': 'doubleclickbidmanager#query',
    'metadata': {
        'dataRange': 'CUSTOM_DATES',
        'format': 'CSV',
        'title': 'JUBA_day'
    },
    'params': {
        'filters': [
            {'type': 'FILTER_ADVERTISER', 'value': '1958747'},
            {'type': 'FILTER_LINE_ITEM', 'value': '13906955'},
            {'type': 'FILTER_LINE_ITEM', 'value': '13857627'},
            {'type': 'FILTER_LINE_ITEM', 'value': '13857390'},
            {'type': 'FILTER_LINE_ITEM', 'value': '13857628'},
        ],
        'groupBys': [
            'FILTER_ADVERTISER',
            'FILTER_LINE_ITEM',
            'FILTER_ACTIVITY_ID',
            'FILTER_DAY_OF_WEEK'
        ],
        'includeInviteData': False,
        'metrics': [
            'METRIC_IMPRESSIONS',
            'METRIC_CLICKS',
            'METRIC_TOTAL_CONVERSIONS',
            'METRIC_REVENUE_ADVERTISER',
            'METRIC_MEDIA_COST_ADVERTISER',
            'METRIC_CTR'
        ],
        'type': 'TYPE_GENERAL'
    },
    'reportDataEndTimeMs': '1510531200000',
    'reportDataStartTimeMs': '1506297600000',
    'schedule': {
        'endTimeMs': '0',
        'frequency': 'ONE_TIME',
        'nextRunTimezoneCode': 'America/New_York'
    }
}
'''

"\nquery_resourse = {\n    'kind': 'doubleclickbidmanager#query',\n    'metadata': {\n        'dataRange': 'CUSTOM_DATES',\n        'format': 'CSV',\n        'title': 'JUBA_day'\n    },\n    'params': {\n        'filters': [\n            {'type': 'FILTER_ADVERTISER', 'value': '1958747'},\n            {'type': 'FILTER_LINE_ITEM', 'value': '13906955'},\n            {'type': 'FILTER_LINE_ITEM', 'value': '13857627'},\n            {'type': 'FILTER_LINE_ITEM', 'value': '13857390'},\n            {'type': 'FILTER_LINE_ITEM', 'value': '13857628'},\n        ],\n        'groupBys': [\n            'FILTER_ADVERTISER',\n            'FILTER_LINE_ITEM',\n            'FILTER_ACTIVITY_ID',\n            'FILTER_DAY_OF_WEEK'\n        ],\n        'includeInviteData': False,\n        'metrics': [\n            'METRIC_IMPRESSIONS',\n            'METRIC_CLICKS',\n            'METRIC_TOTAL_CONVERSIONS',\n            'METRIC_REVENUE_ADVERTISER',\n            'METRIC_MEDIA_COST_ADVERTISER',\n            'METRIC

In [15]:
[service.queries().createquery(body=x).execute()  for x in query_resourse_list]

[{'kind': 'doubleclickbidmanager#query',
  'metadata': {'dataRange': 'CUSTOM_DATES',
   'format': 'CSV',
   'googleCloudStoragePathForLatestReport': '',
   'googleDrivePathForLatestReport': '',
   'latestReportRunTimeMs': '0',
   'reportCount': 0,
   'running': False,
   'sendNotification': False,
   'title': 'LIFT_Date_to_2017-12-28'},
  'params': {'filters': [{'type': 'FILTER_ADVERTISER', 'value': '1958747'},
    {'type': 'FILTER_LINE_ITEM', 'value': '13906955'},
    {'type': 'FILTER_LINE_ITEM', 'value': '13857627'},
    {'type': 'FILTER_LINE_ITEM', 'value': '13857390'},
    {'type': 'FILTER_LINE_ITEM', 'value': '13857628'},
    {'type': 'FILTER_LINE_ITEM', 'value': '13884831'},
    {'type': 'FILTER_LINE_ITEM', 'value': '13857626'},
    {'type': 'FILTER_LINE_ITEM', 'value': '13842564'},
    {'type': 'FILTER_LINE_ITEM', 'value': '13857621'},
    {'type': 'FILTER_LINE_ITEM', 'value': '13788976'},
    {'type': 'FILTER_LINE_ITEM', 'value': '13842563'},
    {'type': 'FILTER_LINE_ITEM', 'v

# list all queries created above

In [16]:
def list_queries():
    '''
    return:
        list of query_id
    '''
    queries = service.queries().listqueries().execute()
    return [x['queryId'] for x in queries['queries']]

In [24]:
queries = service.queries().listqueries().execute()
query_ids = [x['queryId'] for x in queries['queries']]
query_ids

['1514476344336',
 '1514476346048',
 '1514476347629',
 '1514476349268',
 '1514476350938',
 '1514476353020',
 '1514476354845',
 '1514476356530',
 '1514476358243',
 '1514476359861',
 '1514476361722',
 '1514476363665',
 '1514476365390']

In [18]:
# Delete all qury ids
# [service.queries().deletequery(queryId=x).execute() for x in query_ids]

In [19]:
'''
query_ids = list_queries()
query_ids
'''

'\nquery_ids = list_queries()\nquery_ids\n'

In [20]:
# Delete unwanted quries in the service.quries()
# service.queries().deletequery(queryId='1510257342932').execute()

# run queries

In [21]:
def run_queries(query_ids, end_date):
    '''
    Args:
        query_ids, list of str
        end_date, str, eg. '2017-11-08'
    '''
    for query_id in query_ids:
        service.queries().runquery(queryId=query_id, body={
            'dataRange': 'CUSTOM_DATES',
            'reportDataEndTimeMs': date_to_timestampMS(end_date),
            'reportDataStartTimeMs': date_to_timestampMS("2017-09-25"),
            "timezoneCode": 'America/New_York'
        }).execute()

In [22]:
run_queries(query_ids, end_date)

In [28]:
queries = service.queries().listqueries().execute()
queries['queries'][11]

{'kind': 'doubleclickbidmanager#query',
 'metadata': {'dataRange': 'CUSTOM_DATES',
  'format': 'CSV',
  'googleCloudStoragePathForLatestReport': 'https://storage.googleapis.com/136578715968494243751514476363665_report/LIFT_ISP_Carrier_to_2017_12_28_2017-12-28-1553.csv?GoogleAccessId=539605939877-d5f5o3m6kd1mq4j6tl06v7glf07armq9@developer.gserviceaccount.com&Expires=1829836508&Signature=Zvxe7xUQpFBeT9J5y5XGaAsxXR53Q7JA5VYUhaOvGHzUCxtCHso5p3JNVX4wggamKEJ5riM4Iwll084dNtQOasu%2Bvi9jsKOpL629w6Kckw0T7lk9iyzy2J0YDThmaOUg7shIukGSFFisxE6h0bsd2zs279v3qXduAoQNnweP85I%3D',
  'latestReportRunTimeMs': '1514476383547',
  'reportCount': 2,
  'running': False,
  'sendNotification': False,
  'title': 'LIFT_ISP_Carrier_to_2017-12-28'},
 'params': {'filters': [{'type': 'FILTER_ADVERTISER', 'value': '1958747'},
   {'type': 'FILTER_LINE_ITEM', 'value': '13906955'},
   {'type': 'FILTER_LINE_ITEM', 'value': '13857627'},
   {'type': 'FILTER_LINE_ITEM', 'value': '13857390'},
   {'type': 'FILTER_LINE_ITEM', 'val

# download reports

In [29]:
if not os.path.exists('/home/jian/Projects/LIFT/DBM/Data from API/'+till_date+"/"):
    os.makedirs('/home/jian/Projects/LIFT/DBM/Data from API/'+till_date+"/")


In [31]:
download_reports(query_ids, '/home/jian/Projects/LIFT/DBM/Data from API/'+till_date+"/")